Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using DynamicHMCModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_d("..", "scripts", "10")
cd(ProjDir)

### snippet 10.4

In [3]:
d = CSV.read(rel_path("..", "data", "chimpanzees.csv"), delim=';');
df = convert(DataFrame, d);
df[:pulled_left] = convert(Array{Int64}, df[:pulled_left])
df[:prosoc_left] = convert(Array{Int64}, df[:prosoc_left])
df[:condition] = convert(Array{Int64}, df[:condition])
df[:actor] = convert(Array{Int64}, df[:actor])
first(df, 5)

struct m_10_04d_model{TY <: AbstractVector, TX <: AbstractMatrix,
  TA <: AbstractVector}
    "Observations."
    y::TY
    "Covariates"
    X::TX
    "Actors"
    A::TA
    "Number of observations"
    N::Int
    "Number of unique actors"
    N_actors::Int
end

Make the type callable with the parameters *as a single argument*.

In [4]:
function (problem::m_10_04d_model)(θ)
    @unpack y, X, A, N, N_actors = problem   # extract the data
    @unpack β, α = θ  # works on the named tuple too
    ll = 0.0
    ll += sum(logpdf.(Normal(0, 10), β)) # bp & bpC
    ll += sum(logpdf.(Normal(0, 10), α)) # alpha[1:7]
    ll += sum(
      [loglikelihood(Binomial(1, logistic(α[A[i]] + dot(X[i, :], β))), [y[i]]) for i in 1:N]
    )
    ll
end

Instantiate the model with data and inits.

In [5]:
N = size(df, 1)
N_actors = length(unique(df[:actor]))
X = hcat(ones(Int64, N), df[:prosoc_left] .* df[:condition]);
A = df[:actor]
y = df[:pulled_left]
p = m_10_04d_model(y, X, A, N, N_actors);
θ = (β = [1.0, 0.0], α = [-1.0, 10.0, -1.0, -1.0, -1.0, 0.0, 2.0])
p(θ)

-305.21943396408915

Write a function to return properly dimensioned transformation.

In [6]:
problem_transformation(p::m_10_04d_model) =
    as( (β = as(Array, size(p.X, 2)), α = as(Array, p.N_actors), ) )
# Wrap the problem with a transformation, then use Flux for the gradient.
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Tune and sample.

In [7]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.018 s/step ...done
MCMC, adapting ϵ (25 steps)
0.013 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0076 s/step ...done
MCMC, adapting ϵ (100 steps)
0.0053 s/step ...done
MCMC, adapting ϵ (200 steps)
0.0039 s/step ...done
MCMC, adapting ϵ (400 steps)
step 314 (of 400), 0.0032 s/step
0.0031 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0043 s/step ...done
MCMC (1000 steps)
step 289 (of 1000), 0.0035 s/step
step 586 (of 1000), 0.0034 s/step
step 913 (of 1000), 0.0033 s/step
0.0033 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [8]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :α),Tuple{Array{Float64,1},Array{Float64,1}}},1}:
 (β = [0.9148225781253044, 0.6030455424750145], α = [-1.4854625994937818, 7.255614740798381, -1.4791377090435667, -1.5315031669567924, -1.4362141262283483, -0.7521848925996043, 0.38745237968732216])
 (β = [0.6522187773041147, -0.2304448783858327], α = [-0.6423124528367583, 4.553102535145873, -0.9419830859967235, -1.317003945142228, -1.1574662724761648, 0.038294991691608576, 1.009023366224133]) 
 (β = [1.3831972700734636, -0.2684405514481583], α = [-1.4212287677759967, 5.36934124744117, -1.7670854100832114, -1.9011535278388914, -1.9153070579283242, -0.7224100066106176, 0.13297566347591339])
 (β = [3.2653799581688334, 0.7172617210980129], α = [-3.999767642810459, 9.064093255209977, -4.0921851811683965, -4.3755913439841585, -3.9359791398601156, -2.992809564194223, -1.163430990147414])   
 (β = [3.3891830215000027, 0.11013588128180762], α = [-3.815546451894749, 1.5969896206452758, -4.313173261431098, -3.97991

Extract the parameter posterior means: `β`,

In [9]:
posterior_β = mean(first, posterior)

2-element Array{Float64,1}:
 1.388871827275172 
 0.4210063391224993

Extract the parameter posterior means: `β`,

In [10]:
posterior_α = mean(last, posterior)

7-element Array{Float64,1}:
 -1.8325233449830958
 10.148796519402    
 -2.158278503248467 
 -2.1361009317943815
 -1.8368960234337104
 -0.9014016267169456
  0.6459816269775229

Effective sample sizes (of untransformed draws)

In [11]:
ess = mapslices(effective_sample_size, get_position_matrix(chain); dims = 1)
ess

1×9 Array{Float64,2}:
 445.312  1000.0  438.987  516.929  …  458.751  450.268  445.73  437.248

NUTS-specific statistics

In [12]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.91, min/25%/median/75%/max: 0.08 0.89 0.96 0.99 1.0
  termination: AdjacentDivergent => 0% AdjacentTurn => 24% DoubledTurn => 75%
  depth: 2 => 2% 3 => 45% 4 => 53% 5 => 0% 6 => 0%


Result rethinking

In [13]:
rethinking = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
        Mean        SD       Naive SE       MCSE      ESS
a.1 -0.74503184 0.26613979 0.0042080396 0.0060183398 1000
a.2 10.77955494 5.32538998 0.0842018089 0.1269148045 1000
a.3 -1.04982353 0.28535997 0.0045119373 0.0049074219 1000
a.4 -1.04898135 0.28129307 0.0044476339 0.0056325117 1000
a.5 -0.74390933 0.26949936 0.0042611590 0.0052178124 1000
a.6  0.21599365 0.26307574 0.0041595927 0.0045153523 1000
a.7  1.81090866 0.39318577 0.0062168129 0.0071483527 1000
 bp  0.83979926 0.26284676 0.0041559722 0.0059795826 1000
bpC -0.12913322 0.29935741 0.0047332562 0.0049519863 1000
";

Means of draws

In [14]:
[posterior_β, posterior_α]

2-element Array{Array{Float64,1},1}:
 [1.388871827275172, 0.4210063391224993]                                                                                                      
 [-1.8325233449830958, 10.148796519402, -2.158278503248467, -2.1361009317943815, -1.8368960234337104, -0.9014016267169456, 0.6459816269775229]

End of `10/m10.04d.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*